# a recommender system using restricted Boltzmann machines
#### references: Fischer and Igel, 2012, An Introduction to Restricted Boltzmann Machines

In [1]:
# . . import libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data

# . . download the movielens dataset
!wget "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
!unzip ml-100k.zip
!ls

In [2]:
# . . prepare the training and test sets
# . . the separator is tab
train_set = pd.read_csv('ml-100k/u1.base', delimiter='\t')
test_set =  pd.read_csv('ml-100k/u1.test', delimiter='\t')

# . . convert to numpy arrays
train_set = np.array(train_set, dtype='int')
test_set = np.array(test_set, dtype='int')

In [3]:
# . . get the number of users movies and users
# . . take the maximum user id in the training set
num_users  = int(max(max(train_set[:,0], ), max(test_set[:, 0])))
num_movies = int(max(max(train_set[:,1], ), max(test_set[:, 1])))

In [4]:
# . . convert the data into an array where each row represents a user and eadch column represents a movie
def prep_data(data):
    new_data = []
    for id_users in range(1, num_users +1):
        # . . get the movies and ratings belong the current user
        # . . indexes of movies that were rated
        id_movies  = data[:,1] [data[:,0] == id_users]
        # . , the ratings
        id_ratings = data[:,2] [data[:,0] == id_users]
        ratings = np.zeros(num_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [5]:
# . . prepare the training and validation sets
train_set = prep_data(train_set)
test_set  = prep_data(test_set)

# . . transform the tarining and validation sets to Torch tensors
train_set = torch.FloatTensor(train_set)
test_set  = torch.FloatTensor(test_set)

In [6]:
# . . convert the traning ratings into binary form: 1: liked, 0: not liked
# . . zero in the data means it is not rated: make it -1
# . . in the prepared dataset, zero means the user did not like the movie
# . . the training set
train_set[train_set == 0] = -1
train_set[train_set == 1] = 0
train_set[train_set == 2] = 0
train_set[train_set == 3] = 1
# . . the validation set
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set == 3] = 1

In [7]:
# . . the hyperparameters
num_epoch = 10
batch_size = 100
# . . the number of hidden nodes
num_hidden = 100
# . . hte k-steps of the contrasive divergence
num_steps = 10
# . . the fixed parameters
# . . the number of visible nodes: the number of movies
num_visible = len(train_set[0])


In [8]:
# . . the restricted Boltzmann machine: an energy-based model
# . . a restricted Boltzmann machine is a probabilistic graphical model
class RBM():
    def __init__(self, num_visible, num_hidden):
        # . . the weight matrix
        self.W = torch.randn(num_hidden, num_visible)
        # . . the bias of the hidden nodes
        self.a = torch.randn(1, num_hidden)
        # . . the bias of the visible nodes
        self.b = torch.randn(1, num_visible)

    # . . compute the probability of the hidden nodes given the visible nodes
    def sample_hidden(self, v):
        # . . W @ v
        wv  = torch.mm(v, self.W.t())
        
        # . . W @ v + a
        wva = wv + self.a.expand_as(wv)        

        # . . the probability of the hidden node will be activated given the visible node
        prob_hidden = torch.sigmoid(wva)

        # . . draw a binary random number (0 or 1) from a Bernoulli distribution
        # . . to decide whether to activate the hidden neuron or not according to its probability
        sample_hidden = torch.bernoulli(prob_hidden)

        return prob_hidden, sample_hidden

    # . . compute the probability of the visible nodes given the hidden nodes
    def sample_visible(self, h):
        # . . W @ y
        wh  = torch.mm(h, self.W)
        
        # . . W @ h + b
        whb = wh + self.b.expand_as(wh)
        
        # . . the probability of the hidden will be activated node given the visible node
        prob_visible = torch.sigmoid(whb)

        # . . draw a binary random number (0 or 1) from a Bernoulli distribution
        # . . to decide whether to activate the visible neuron or not according to its probability
        sample_visible = torch.bernoulli(prob_visible)

        return prob_visible, sample_visible


    # . . train the RBM with the contrasive divergence
    # . . the contrasive divergence approximates the log likelihood gradient
    # . . we are updating weights to minimazie the energy (i.e., maximize the log lokelihood)
    # . . v0: vector of observations: movie ratings by the user
    # . . vk: visible nodes obtained after k sampling (kth-step contrasive divergence)
    # . . ph0: vector of prior probabilities of hidden given the movie ratings (v0)
    # . . phk: vector of probabilities of hidden given the movie ratings (vk) after k-sampling
    def train(self, v0, vk, ph0, phk):
        # . .step 8,9 , and 10 of the paper (see the reference at top)
        # . . update the weights
        self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()                
        # . . update the bias of visible nodes
        self.b += torch.sum((v0 - vk), 0)
        # . . update the bias of hidden nodes
        self.a += torch.sum((ph0 - phk), 0)

In [9]:
# . . instantiate the model
rbm = RBM(num_visible, num_hidden)

In [10]:
# . . train the RBM with k-step contrasive divergence
for epoch in range(1, num_epoch + 1):
    train_loss = 0
    loss = []
    batch = 0
    for id_user in range(0, num_users - batch_size, batch_size):    
        # . . do not change the targets: v0
        v0 = train_set[id_user: id_user + batch_size]
        vk = train_set[id_user: id_user + batch_size]
        
        ph0, _ = rbm.sample_hidden(v0)
        # . . the k-steps of the contrasive divergence: random walk (Gibbs sampling)
        for k in range(num_steps):
          _, hk = rbm.sample_hidden(vk)
          _, vk = rbm.sample_visible(hk)
          # . . do no include the -1 ratings (movies theuser didn't watch) in the training
          vk[v0<0] = v0[v0<0]
        phk, _ = rbm.sample_hidden(vk)
        rbm.train(v0, vk, ph0, phk)
        # . . the free energy of the system: (representative)
        # . . only include the existing ratings in the loss
        train_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0]))
        batch += 1.
    print('Epoch: '+str(epoch)+' Train loss: '+str(train_loss/batch))

Epoch: 1 Train loss: tensor(2.1906)
Epoch: 2 Train loss: tensor(2.1173)
Epoch: 3 Train loss: tensor(2.1135)
Epoch: 4 Train loss: tensor(2.1134)
Epoch: 5 Train loss: tensor(2.1133)
Epoch: 6 Train loss: tensor(2.1149)
Epoch: 7 Train loss: tensor(2.1145)
Epoch: 8 Train loss: tensor(2.1150)
Epoch: 9 Train loss: tensor(2.1134)
Epoch: 10 Train loss: tensor(2.1126)


In [11]:
# . . test the trained network
test_loss = 0
batch = 0.
for id_user in range(num_users):    
    # . . do not change the targets: vt
    # . . predict users sequentially
    v  = train_set[id_user: id_user + 1]
    vt = test_set[ id_user: id_user + 1]
    
    # . . we only need a single step: this is a blind walk
    # . . take only the existing ratings
    if len(vt[vt >= 0]) > 0:
      _, h = rbm.sample_hidden(v)
      _, v = rbm.sample_visible(h)

      # . . the free energy of the system: (representative)
      # . . only include the existing ratings in the loss
      test_loss += torch.mean(torch.abs(vt[vt >= 0] - v[vt >= 0]))
      # . . update the batch counter
      batch += 1.
print('Test loss: '+str(test_loss/batch))

Test loss: tensor(2.2351)
